In [1]:
import pandas as pd
import numpy as np

# 1. Read your CSV
df = pd.read_csv("/media/sagarkumar/New Volume/SAGAR/IPYNB_FILE/DATA_GENERATION/final_two_column_with_rank_11_withoutDT.csv")

# 2. Compute the constant denominator: 11 * 0.95 * √3
den = 11 * 0.95 * np.sqrt(3)

# 3. Create the new column
df["DT_LOAD_CAL"] = df["DT_LOAD"] / den

# 4. (Optional) inspect first few rows
print(df[["DT_LOAD", "DT_LOAD_CAL"]].head())

# 5. Save to a new CSV (or overwrite)
df.to_csv("/media/sagarkumar/New Volume/SAGAR/IPYNB_FILE/DATA_GENERATION/fedder_11KV_with_DT_LOAD_CAL.csv", index=False)


      DT_LOAD  DT_LOAD_CAL
0  127.425882     7.040131
1  233.628927    12.907725
2  134.062123     7.406775
3  245.587090    13.568399
4  364.485990    20.137424


MAP THE LOAD OF MEANHLY AVERGAE WITH THE FEEDER

In [ ]:
import pandas as pd

# 1) Load your files
sw = pd.read_csv("/media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/CSV FILE/Monthly_average_from_scada/monthly_SWNO_matrix_numpy.csv")       # contains SWNO, Month_01…Month_12
feeder = pd.read_csv("/media/sagarkumar/New Volume/SAGAR/IPYNB_FILE/DATA_GENERATION/fedder_11KV_with_DT_LOAD_CAL.csv")      # contains feeder_id, DT_LOAD_CAL

# 2) Build a lookup from feeder_id → DT_LOAD_CAL
dt_map = feeder.set_index("FEEDER_ID")["DT_LOAD_CAL"].to_dict()

# 3) Prepare for the walk:
#    We'll keep track of the "current" DT_LOAD_CAL and the last calculated values for each month.
current_dt = None
prev_vals = {f"Month_{i:02d}": None for i in range(1,13)}

# 4) Iterate rows in order, computing chain‐subtract columns:
out_rows = []
for _, row in sw.iterrows():
    swno = row["SWNO"]
    # If this SWNO is a feeder anchor, reset current_dt and prev_vals
    if swno in dt_map:
        current_dt = dt_map[swno]
        # initialize prev_vals for this anchor: first subtraction
        for i in range(1,13):
            m = f"Month_{i:02d}"
            prev_vals[m] = row[m] - current_dt
    else:
        # continue chain: subtract the same current_dt from last prev_vals
        for i in range(1,13):
            m = f"Month_{i:02d}"
            prev_vals[m] = prev_vals[m] - current_dt

    # build output row
    out = row.to_dict()
    # add the new _CAL columns
    for i in range(1,13):
        m = f"Month_{i:02d}"
        out[f"{m}_CAL"] = prev_vals[m]
    out_rows.append(out)

# 5) Assemble and write out
out_df = pd.DataFrame(out_rows)
out_df.to_csv("/media/sagarkumar/New Volume/SAGAR/IPYNB_FILE/DATA_GENERATION/feeder_11Kv_sw_with_chain_calculations.csv", index=False)


COMMULATIVE DIFFERENCE FOR MONTHLY AVERAGE LOAD OF SCADA


In [3]:
import pandas as pd

# File paths
feeder_path = "/media/sagarkumar/New Volume/SAGAR/IPYNB_FILE/DATA_GENERATION/fedder_11KV_with_DT_LOAD_CAL.csv"
monthly_path = "/media/sagarkumar/New Volume/SAGAR/IPYNB_FILE/CSV FILE/Monthly_average_from_scada/monthly_SWNO_matrix_numpy.csv"
output_path = "/media/sagarkumar/New Volume/SAGAR/IPYNB_FILE/DATA_GENERATION/final_feeder_11kv_monthly_cumulative_diff.csv"


# Load data
feeder_df = pd.read_csv(feeder_path)
monthly_df = pd.read_csv(monthly_path)

# Merge on FEEDER_ID <-> SWNO
merged = feeder_df.merge(monthly_df, left_on='FEEDER_ID', right_on='SWNO', how='left')

month_cols = [f'Month_{i:02d}' for i in range(1, 13)]

result_rows = []

for feeder_id, group in merged.groupby('FEEDER_ID', sort=False):
    group = group.copy()
    group['__orig_idx'] = group.index
    ranks_in_order = group['RANK'].drop_duplicates().tolist()

    dt_load_sums = {
        rank: group[group['RANK'] == rank]['DT_LOAD_CAL'].fillna(0).sum()
        for rank in ranks_in_order
    }

    prev_cum = {m: None for m in month_cols}

    for rank in ranks_in_order:
        this_rank_rows = group[group['RANK'] == rank].sort_values('__orig_idx')
        first = True
        for idx, row in this_rank_rows.iterrows():
            base = {col: row[col] for col in feeder_df.columns}
            for m in month_cols:
                month_val = row.get(m, None)
                if first:
                    # Only for the first row of this rank
                    if rank == ranks_in_order[0]:
                        cum_val = month_val - dt_load_sums[rank] if pd.notna(month_val) else None
                    else:
                        prev = prev_cum[m]
                        cum_val = prev - dt_load_sums[rank] if prev is not None else None
                    prev_cum[m] = cum_val
                else:
                    cum_val = None  # For subsequent rows at same rank, leave blank
                base[f"{m}_cumulative_diff"] = cum_val
            result_rows.append(base)
            first = False

result_df = pd.DataFrame(result_rows)
output_cols = list(feeder_df.columns) + [f"{m}_cumulative_diff" for m in month_cols]
result_df.to_csv(output_path, index=False, columns=output_cols)
print(f"Saved to {output_path}")


Saved to /media/sagarkumar/New Volume/SAGAR/IPYNB_FILE/DATA_GENERATION/final_feeder_11kv_monthly_cumulative_diff.csv


AFTER THE RANKING AND USE ONLY MAIN CHAIN DONT USING SUBCHAIN

In [1]:
import pandas as pd
import numpy as np

# 1. Read your CSV
df = pd.read_csv("/media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/DATA_GENERATION/11_KV_FEEDER_ALL DATA/11_WITHOUTDT_CONNECTED_RANKED.csv")

# 2. Compute the constant denominator: 11 * 0.95 * √3
den = 11 * 0.95 * np.sqrt(3)

# 3. Create the new column
df["DT_LOAD_CAL"] = df["DT_LOAD"] / den

# 4. (Optional) inspect first few rows
print(df[["DT_LOAD", "DT_LOAD_CAL"]].head())

# 5. Save to a new CSV (or overwrite)
df.to_csv("/media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/DATA_GENERATION/11_KV_FEEDER_ALL DATA/fedder_11KV_with_DT_LOAD_CAL.csv", index=False)


      DT_LOAD  DT_LOAD_CAL
0  616.320000    34.050959
1  674.080488    37.242158
2  473.442574    26.157148
3  148.855367     8.224085
4  400.698507    22.138124


In [3]:
import pandas as pd

# 1) Load your files
sw = pd.read_csv("/media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/CSV FILE/Monthly_average_from_scada/monthly_SWNO_matrix_numpy.csv")       # contains SWNO, Month_01…Month_12
feeder = pd.read_csv("/media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/DATA_GENERATION/11_KV_FEEDER_ALL DATA/fedder_11KV_with_DT_LOAD_CAL.csv")      # contains feeder_id, DT_LOAD_CAL

# 2) Build a lookup from feeder_id → DT_LOAD_CAL
dt_map = feeder.set_index("FEEDER_ID")["DT_LOAD_CAL"].to_dict()

# 3) Prepare for the walk:
#    We'll keep track of the "current" DT_LOAD_CAL and the last calculated values for each month.
current_dt = None
prev_vals = {f"Month_{i:02d}": None for i in range(1,13)}

# 4) Iterate rows in order, computing chain‐subtract columns:
out_rows = []
for _, row in sw.iterrows():
    swno = row["SWNO"]
    # If this SWNO is a feeder anchor, reset current_dt and prev_vals
    if swno in dt_map:
        current_dt = dt_map[swno]
        # initialize prev_vals for this anchor: first subtraction
        for i in range(1,13):
            m = f"Month_{i:02d}"
            prev_vals[m] = row[m] - current_dt
    else:
        # continue chain: subtract the same current_dt from last prev_vals
        for i in range(1,13):
            m = f"Month_{i:02d}"
            prev_vals[m] = prev_vals[m] - current_dt

    # build output row
    out = row.to_dict()
    # add the new _CAL columns
    for i in range(1,13):
        m = f"Month_{i:02d}"
        out[f"{m}_CAL"] = prev_vals[m]
    out_rows.append(out)

# 5) Assemble and write out
out_df = pd.DataFrame(out_rows)
out_df.to_csv("/media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/DATA_GENERATION/11_KV_FEEDER_ALL DATA/feeder_11Kv_sw_with_chain_calculations.csv", index=False)


In [1]:
import pandas as pd

# File paths (change as needed)
feeder_path = "/media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/DATA_GENERATION/11_KV_FEEDER_ALL DATA/fedder_11KV_with_DT_LOAD_CAL.csv"
daily_path_1 = "/media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/DATA_GENERATION/11_KV_FEEDER_ALL DATA/daily_SWNO_matrix_11KV_YEAR2025.csv"
daily_path_2 = "/media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/DATA_GENERATION/11_KV_FEEDER_ALL DATA/daily_SWNO_matrix_11KV_YEAR2024.csv"
output_path_1 = "/media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/DATA_GENERATION/11_KV_FEEDER_ALL DATA/final_feeder_11kv_daily_cumulative_diff_2025.csv"
output_path_2 = "/media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/DATA_GENERATION/11_KV_FEEDER_ALL DATA/final_feeder_11kv_daily_cumulative_diff_2024.csv"


In [2]:
feeder_df = pd.read_csv(feeder_path)

def is_main_chain_only(ranks):
    return all('.' not in str(r) for r in ranks)

main_chain_flags = (
    feeder_df.groupby('FEEDER_ID')['RANK']
    .apply(is_main_chain_only)
    .rename('IS_MAIN_CHAIN_ONLY')
    .reset_index()
)
main_chain_ids = main_chain_flags[main_chain_flags['IS_MAIN_CHAIN_ONLY']]['FEEDER_ID'].tolist()
main_chain_feeder_df = feeder_df[feeder_df['FEEDER_ID'].isin(main_chain_ids)].copy()


In [3]:
def process_year(main_chain_feeder_df, daily_path, output_path):
    daily_df = pd.read_csv(daily_path)
    day_cols = [col for col in daily_df.columns if col.startswith("Day_")]

    merged = main_chain_feeder_df.merge(daily_df, left_on='FEEDER_ID', right_on='SWNO', how='left')

    result_rows = []

    for feeder_id, group in merged.groupby('FEEDER_ID', sort=False):
        group = group.copy()
        group['__orig_idx'] = group.index
        ranks_in_order = group['RANK'].drop_duplicates().tolist()

        dt_load_sums = {
            rank: group[group['RANK'] == rank]['DT_LOAD_CAL'].fillna(0).sum()
            for rank in ranks_in_order
        }

        prev_cum = {d: None for d in day_cols}

        for rank in ranks_in_order:
            this_rank_rows = group[group['RANK'] == rank].sort_values('__orig_idx')
            first = True
            for idx, row in this_rank_rows.iterrows():
                base = {col: row[col] for col in feeder_df.columns}
                for d in day_cols:
                    day_val = row.get(d, None)
                    if first:
                        if rank == ranks_in_order[0]:
                            cum_val = day_val - dt_load_sums[rank] if pd.notna(day_val) else None
                        else:
                            prev = prev_cum[d]
                            cum_val = prev - dt_load_sums[rank] if prev is not None else None
                        prev_cum[d] = cum_val
                    else:
                        cum_val = None
                    base[f"{d}_cumulative_diff"] = cum_val
                result_rows.append(base)
                first = False

    result_df = pd.DataFrame(result_rows)
    output_cols = list(feeder_df.columns) + [f"{d}_cumulative_diff" for d in day_cols]
    result_df.to_csv(output_path, index=False, columns=output_cols)
    print(f"Saved to {output_path}")

# Process each year
process_year(main_chain_feeder_df, daily_path_1, output_path_1)
process_year(main_chain_feeder_df, daily_path_2, output_path_2)


Saved to /media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/DATA_GENERATION/11_KV_FEEDER_ALL DATA/final_feeder_11kv_daily_cumulative_diff_2025.csv
Saved to /media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/DATA_GENERATION/11_KV_FEEDER_ALL DATA/final_feeder_11kv_daily_cumulative_diff_2024.csv


In [4]:
import pandas as pd

# --- Identify only main-chain feeders (no sub-branches) ---
feeder_df = pd.read_csv(feeder_path)

def is_main_chain_only(ranks):
    return all('.' not in str(r) for r in ranks)

main_chain_flags = (
    feeder_df.groupby('FEEDER_ID')['RANK']
    .apply(is_main_chain_only)
    .rename('IS_MAIN_CHAIN_ONLY')
    .reset_index()
)
main_chain_ids = main_chain_flags[main_chain_flags['IS_MAIN_CHAIN_ONLY']]['FEEDER_ID'].tolist()
main_chain_feeder_df = feeder_df[feeder_df['FEEDER_ID'].isin(main_chain_ids)].copy()

# --- Processing function ---
def process_year(main_chain_feeder_df, daily_path, output_path):
    daily_df = pd.read_csv(daily_path)
    day_cols = [col for col in daily_df.columns if col.startswith("Day_")]

    merged = main_chain_feeder_df.merge(daily_df, left_on='FEEDER_ID', right_on='SWNO', how='left')

    result_rows = []

    for feeder_id, group in merged.groupby('FEEDER_ID', sort=False):
        group = group.copy()
        group['__orig_idx'] = group.index
        ranks_in_order = group['RANK'].drop_duplicates().tolist()

        dt_load_sums = {
            rank: group[group['RANK'] == rank]['DT_LOAD_CAL'].fillna(0).sum()
            for rank in ranks_in_order
        }

        prev_cum = {d: None for d in day_cols}

        for rank in ranks_in_order:
            this_rank_rows = group[group['RANK'] == rank].sort_values('__orig_idx')
            first = True
            for idx, row in this_rank_rows.iterrows():
                base = {col: row[col] for col in main_chain_feeder_df.columns}
                for d in day_cols:
                    day_val = row.get(d, None)
                    if first:
                        if rank == ranks_in_order[0]:
                            cum_val = day_val - dt_load_sums[rank] if pd.notna(day_val) else None
                        else:
                            prev = prev_cum[d]
                            cum_val = prev - dt_load_sums[rank] if prev is not None else None
                        prev_cum[d] = cum_val
                    else:
                        cum_val = None
                    base[f"{d}_cumulative_diff"] = cum_val
                result_rows.append(base)
                first = False

    result_df = pd.DataFrame(result_rows)
    # --- Add FROM_SWITCH and TO_SWITCH columns ---
    if "FROM_TO" in result_df.columns:
        from_to_split = result_df["FROM_TO"].str.split("-", n=1, expand=True)
        result_df["FROM_SWITCH"] = from_to_split[0]
        result_df["TO_SWITCH"] = from_to_split[1]
        cols = list(result_df.columns)
        i = cols.index("FROM_TO") + 1
        for col in ["FROM_SWITCH", "TO_SWITCH"]:
            cols.insert(i, cols.pop(cols.index(col)))
            i += 1
        result_df = result_df[cols]
    # --- Save ---
    output_cols = list(result_df.columns)  # Save all columns, now includes FROM_SWITCH and TO_SWITCH
    result_df.to_csv(output_path, index=False, columns=output_cols)
    print(f"Saved to {output_path}")

# --- Process each year ---
process_year(main_chain_feeder_df, daily_path_1, output_path_1)
process_year(main_chain_feeder_df, daily_path_2, output_path_2)


Saved to /media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/DATA_GENERATION/11_KV_FEEDER_ALL DATA/final_feeder_11kv_daily_cumulative_diff_2025.csv
Saved to /media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/DATA_GENERATION/11_KV_FEEDER_ALL DATA/final_feeder_11kv_daily_cumulative_diff_2024.csv
